In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
loan_data_processed = pd.read_csv("Data/loan_data_2007_2014_preprocessed.csv")
loan_data_processed.head()

,Unnamed: 0,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,...,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,good_bad
0,0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,...,0,0,0,0,0,0,0,1,0,1
1,1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,...,0,0,0,0,0,0,0,1,0,0
2,2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,...,0,0,0,0,0,0,0,1,0,1
3,3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,...,0,0,0,0,0,0,0,1,0,1
4,4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,...,0,0,0,0,0,0,0,1,0,1


In [4]:
#missing values
pd.options.display.max_rows=None
loan_data_processed.isnull().sum()

Unnamed: 0                                                              0
Unnamed: 0.1                                                            0
id                                                                      0
member_id                                                               0
loan_amnt                                                               0
funded_amnt                                                             0
funded_amnt_inv                                                         0
term                                                                    0
int_rate                                                                0
installment                                                             0
grade                                                                   0
sub_grade                                                               0
emp_title                                                           27588
emp_length                            

In [5]:
#handle missing values
loan_data_processed['mths_since_last_delinq'].fillna(0,inplace=True)

In [6]:
loan_data_processed['mths_since_last_record'].fillna(0,inplace=True)

In [7]:
#features we are using for lgd and ead model
features_all= ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:MORTGAGE',
'home_ownership:NONE',
'home_ownership:OTHER',
'home_ownership:OWN',
'home_ownership:RENT',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:car',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:educational',
'purpose:home_improvement',
'purpose:house',
'purpose:major_purchase',
'purpose:medical',
'purpose:moving',
'purpose:other',
'purpose:renewable_energy',
'purpose:small_business',
'purpose:vacation',
'purpose:wedding',
'initial_list_status:f',
'initial_list_status:w',
'term_int',
'emp_length_int',
'mths_since_issue_d',
'mths_since_earliest_cr_line',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_delinq',
'mths_since_last_record',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim']

In [8]:
features_reference_cat=['grade:G',
'home_ownership:RENT',
'verification_status:Verified',
'purpose:credit_card',
'initial_list_status:f']

In [9]:
#Now we will use the data for EAD and LGD will same but it differs for PD model
loan_data_processed_lgd_ead = loan_data_processed[features_all]
loan_data_processed_lgd_ead = loan_data_processed_lgd_ead.drop(features_reference_cat, axis=1)

In [28]:
#load all the pickle files of the models
pd_lr=pickle.load(open("pd_model.sav","rb"))
lgd_s1=pickle.load(open("lgd_model_stage_1.sav","rb"))
lgd_s2=pickle.load(open("lgd_model_stage_2.sav","rb"))
ead=pickle.load(open("ead_model.sav","rb"))


In [29]:
loan_data_processed['recovery_rate_st_1'] = lgd_s1.predict(loan_data_processed_lgd_ead)

In [13]:
loan_data_processed['recovery_rate_st_2'] = lgd_s2.predict(loan_data_processed_lgd_ead)

In [14]:
#recovery rate we have to multiply both stage1 and stage2
loan_data_processed['recovery_rate'] = loan_data_processed['recovery_rate_st_1']*loan_data_processed['recovery_rate_st_2']

In [16]:
# As we know that linear regression model can predict the recovery rate 
#<0 or >1 but this should be in interval [0,1]. So again we will replace as 0
# in palce of <0 and 1 inplace of >1.
loan_data_processed['recovery_rate']= np.where(loan_data_processed['recovery_rate']<0,0,loan_data_processed['recovery_rate'])
loan_data_processed['recovery_rate']= np.where(loan_data_processed['recovery_rate']>1,1,loan_data_processed['recovery_rate'])

In [18]:
# LGD= 1- recovery rate
loan_data_processed['LGD'] = 1 - loan_data_processed['recovery_rate']


In [19]:
loan_data_processed['LGD'].describe()

count    466285.000000
mean          0.929272
std           0.058756
min           0.296372
25%           0.878110
50%           0.910304
75%           1.000000
max           1.000000
Name: LGD, dtype: float64

In [20]:
#for EAD we will use CCF
loan_data_processed['CCF']= ead.predict(loan_data_processed_lgd_ead)

In [21]:
# As we know that linear regression model can predict the recovery rate 
#<0 or >1 but this should be in interval [0,1]. So again we will replace as 0
# in palce of <0 and 1 inplace of >1.
loan_data_processed['CCF']= np.where(loan_data_processed['CCF']<0,0,loan_data_processed['CCF'])
loan_data_processed['CCF']= np.where(loan_data_processed['CCF']>1,1,loan_data_processed['CCF'])

In [22]:
# as we know that EAD= funded amount * credit conversion factor(CCF)
loan_data_processed['EAD']= loan_data_processed['CCF'] * loan_data_processed['funded_amnt']


In [23]:
#describe EAD
loan_data_processed['EAD'].describe()

count    466285.000000
mean      10800.542943
std        6924.760825
min         190.163139
25%        5491.514690
50%        9198.605320
75%       14669.677941
max       35000.000000
Name: EAD, dtype: float64

In [32]:
#use loan_data_train and loan_data_test 
loan_data_train=pd.read_csv("Data/loan_data_inputs_train.csv")
loan_data_test= pd.read_csv("Data/loan_data_inputs_test.csv")

In [33]:
loan_data_inputs_pd= pd.concat([loan_data_train,loan_data_test],axis=0)
loan_data_inputs_pd.head()

,Unnamed: 0,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
0,327896,327896,23964931,26337651,12000,12000,12000.0,36,16.29,423.61,...,0,0,0,0,0,0,0,0,1,0
1,237691,237691,36261200,38972610,19800,19800,19800.0,60,17.86,501.29,...,0,0,0,1,0,0,0,0,0,0
2,249332,249332,34884391,37567722,2000,2000,2000.0,36,12.99,67.38,...,0,0,0,0,0,0,0,1,0,0
3,88093,88093,7082975,8744579,16425,16425,16375.0,60,12.99,373.64,...,0,0,0,1,0,0,0,0,0,0
4,217265,217265,1284896,1528668,8500,8500,8500.0,36,15.31,295.95,...,0,0,0,1,0,0,0,0,0,0


In [34]:
loan_data_inputs_pd = loan_data_inputs_pd.set_index("Unnamed: 0")

In [35]:
loan_data_inputs_pd.head()

,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
327896,327896,23964931,26337651,12000,12000,12000.0,36,16.29,423.61,D,...,0,0,0,0,0,0,0,0,1,0
237691,237691,36261200,38972610,19800,19800,19800.0,60,17.86,501.29,D,...,0,0,0,1,0,0,0,0,0,0
249332,249332,34884391,37567722,2000,2000,2000.0,36,12.99,67.38,C,...,0,0,0,0,0,0,0,1,0,0
88093,88093,7082975,8744579,16425,16425,16375.0,60,12.99,373.64,B,...,0,0,0,1,0,0,0,0,0,0
217265,217265,1284896,1528668,8500,8500,8500.0,36,15.31,295.95,C,...,0,0,0,1,0,0,0,0,0,0


In [44]:
# Variables
features_all_pd= ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86',
]

In [45]:
ref_categories_pd = ['grade:G',
'home_ownership:RENT',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [46]:
loan_data_inputs_pd_temp = loan_data_inputs_pd[features_all_pd]
loan_data_inputs_pd_temp = loan_data_inputs_pd_temp.drop(ref_categories_pd,axis=1)

In [47]:
loan_data_inputs_pd_temp.shape

(466285, 84)

In [48]:
prob_default=pd_lr.predict_proba(loan_data_inputs_pd_temp)[:][:,0]

In [49]:
prob_default

array([0.1061808 , 0.15014263, 0.07959428, ..., 0.18133069, 0.09558824,
       0.12953903])

In [51]:
loan_data_inputs_pd['PD'] = pd_lr.predict_proba(loan_data_inputs_pd_temp)[:][:,0]

In [52]:
#describe the PD feature
loan_data_inputs_pd['PD'].describe()

count    466285.000000
mean          0.109061
std           0.069089
min           0.008464
25%           0.056645
50%           0.094333
75%           0.146093
max           0.689655
Name: PD, dtype: float64

## calculation of expected loss


In [53]:
loan_data_processed_new = pd.concat([loan_data_processed,loan_data_inputs_pd],axis=1)

In [55]:
loan_data_processed_new['EL']= loan_data_processed_new['PD']*loan_data_processed_new['LGD']*loan_data_processed_new['EAD']

In [56]:
loan_data_processed_new[['funded_amnt','PD','EAD','EL']].head()

,funded_amnt,funded_amnt,PD,EAD,EL
0,5000,5000,0.135016,2936.324988,362.876752
1,2500,2500,0.264265,1922.837542,465.468790
2,2400,2400,0.209115,1569.045958,300.673047
3,10000,10000,0.175071,6662.937189,1058.046758
4,3000,3000,0.108316,2131.489765,211.273241


In [57]:
# sum of expected loss of all customes
loan_data_processed_new['EL'].sum()

510806743.12112784

In [58]:
#sum of all funded amount of all the customers
loan_data_processed_new['funded_amnt'].sum()

funded_amnt    6664052450
funded_amnt    6664052450
dtype: int64

In [59]:
# percentage of epected loss 
loan_data_processed_new['EL'].sum()/ loan_data_processed_new['funded_amnt'].sum()

funded_amnt    0.076651
funded_amnt    0.076651
dtype: float64

A bank holds 10% of its assets as capital
The EL on its portfolio should be less than its capital.
Observed EL values are anywhere between 2% and 10%.
